In [17]:
import logging
import sys
from io import open
from os import path
from time import time
from glob import glob
from gensim.models import Word2Vec
from six.moves import range, zip, zip_longest
from six import iterkeys
from collections import defaultdict, Iterable
import random
from random import shuffle
from itertools import product, permutations
from scipy.io import loadmat
from scipy.sparse import issparse
from io import open
import deepwalk
import pandas as pd

In [14]:
# インプットするファイル名
INPUT = "p2p-Gnutella08.edgelist"

# アウトプットするファイル名
OUTPUT = "p2p-Gnutella08.txt"
# １ノードあたりに実行するウォーク数
NUMBER_WALKS = 5

# 1ウォークあたりの長さ
WALK_LENGTH = 20

# ランダムシード値
SEED = 0

# 埋め込み後の各ノードの次元数
REPRESENTATION_SIZE = 2

# skipgramので使われるウィンドウサイズ
WINDOW_SIZE = 5

# 並列プロセス数
WORKERS = 1


In [25]:
"""
    グラフ生成
    G = load_edgelist(ファイル名:String, 無効グラフかどうか:boolean)
    """
G = deepwalk.load_edgelist(INPUT, True)
# ノード数出力
print("Number of nodes: {}".format(len(G.nodes())))

# ウォーク数算出(ノード数*1ノードあたりのウォーク数)と出力
num_walks = len(G.nodes()) * NUMBER_WALKS
print("Number of walks: {}".format(num_walks))

# データサイズ(ウォーク数*ウォークの長さ)の算出と出力
data_size = num_walks * WALK_LENGTH
print("Data size (walks*length): {}".format(data_size))

print("Walking Now!!!!!!")

# ウォーク取得(長さは全部等しい)
walks = deepwalk.build_deepwalk_corpus(G, num_paths=NUMBER_WALKS,
                              path_length=WALK_LENGTH, alpha=0, rand=random.Random(SEED))

print("Embedding Now!!!!!")
model = Word2Vec(walks, size=REPRESENTATION_SIZE,
                 window=WINDOW_SIZE, min_count=0, sg=1, hs=1, workers=WORKERS)

model.wv.save_word2vec_format(OUTPUT)

#全ベクトルを取得
vec = model.wv.__getitem__([str(i) for i in range(len(G))])


Number of nodes: 6301
Number of walks: 31505
Data size (walks*length): 630100
Walking Now!!!!!!
Embedding Now!!!!!


In [29]:
df = pd.DataFrame(data = vec)

array([[-1.8784282 , -2.8028502 ],
       [-1.6424943 , -2.5639262 ],
       [-1.3810443 , -1.9303118 ],
       ...,
       [-1.2122581 ,  0.7113602 ],
       [-1.0416414 ,  0.54290986],
       [-1.2060262 ,  0.9566122 ]], dtype=float32)